In [42]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
import scipy.stats as stats

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    derived_data = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    derived_data= '/Users/alina/Desktop/MIT/code/data/output/derived_data'
print(derived_data)
#os.listdir(data_root)

/Users/alina/Desktop/MIT/code/data/output/derived_data


In [44]:
baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}

version_form = ['version_form']

qsts = ['snap', 'ssrs',  'pc']##, 'wechsler'] #masc to many missing data 

In [45]:
snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
#masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group_file = 'treatment_groups.csv'
#outcome variablles 
snap_vars = ['snainatx', 'snahypax', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
#masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
#wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars,  'pc': pc_vars} #, 'wechsler': wechsler_vars}

interaction_predictors = ['days_baseline', 'site', 'trtname'] #time, site, treatment group

# mediator variables
comorb_mediators  = ['cdorodd' , 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa'] #ODD/CD or anx excluding specific phobia 
services_mediators =  ['demo61'] #reciept of public assistance 
prev_med_mediators = ['hi_24'] #medication intake prior to study 

#moderator variables 
accept_moderator = ['d2dresp'] # initail acceptance of treatment 
raters = ['Teacher', 'Parent']

treat_group = pd.read_csv(Path(derived_data, treat_group_file))

In [47]:
snap = prep.get_data(Path(data_root, snap_file), columns= [baseline_var, snap_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
ssrs = prep.get_data(Path(data_root, ssrs_file), columns= [baseline_var, ssrs_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
pc = prep.get_data(Path(data_root, parent_child_file), columns= [baseline_var, pc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)
#wechsler = prep.get_data(Path(data_root, wechsler_file), columns= [baseline_var, wechsler_vars], set_dtypes= True, version_form= False, split_timepoints= True)
data_dict = dict(zip(qsts, [snap, ssrs, pc]))

Success
Success
Success


In [48]:
print(data_dict.keys())
print(data_dict['snap'].keys())

dict_keys(['snap', 'ssrs', 'pc'])
dict_keys(['b', '14', '24', '36'])


In [178]:
results_table  = pd.read_csv(Path(derived_data, 'original_and_new_rr_14.csv'))
results_table

,Unnamed: 0.1,Unnamed: 0,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F_value,replicated P_value,variable
0,0,0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,16.06,0.00,snainatx
1,1,1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,.56,1.47,0.11,snainatx
2,2,2,ADHD symptoms,Inattention,Teacher,Site,2.70,.02,0.29,0.92,snainatx
3,3,3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,11.42,0.00,snainatx
4,4,4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,.88,0.78,0.70,snainatx
5,5,5,ADHD symptoms,Inattention,Parent,Site,3.00,.02,0.48,0.79,snainatx
6,6,6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,11.42,0.00,snahypax
7,7,7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,.49,0.78,0.70,snahypax
8,8,8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,.02,0.48,0.79,snahypax
9,9,9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,21.57,0.00,snahypax


In [50]:

# results_table = results_table.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

# results_table_cols_order = results_table.columns
# index_m_r = results_table.columns.get_loc('Measure and Rater')
# index_m_r

# results_table.loc[results_table['Measure and Rater'] == 'SSRS Teacher', 'Measure and Rater'] = 'SSRS - Teacher'
# results_table.loc[results_table['Measure and Rater'] == 'SSRS Parent', 'Measure and Rater'] = 'SSRS - Parent'

# results_table[['Measure', 'Rater']] = results_table['Measure and Rater'].str.split(' - ', expand=True)
# results_table = results_table.drop(columns=['Measure and Rater'])

# results_table.insert(index_m_r, 'Measure', results_table.pop('Measure'))
# results_table.insert(index_m_r + 1, 'Rater', results_table.pop('Rater'))
# results_table

### Contrasts 1 : \
- (M+C) vs (P + CC)
- M vs C
- B vs CC

In [51]:
def get_contrast_matrix(paper = 'original', new_order = None):
    if paper == 'original':
        
        contrast_matrix = np.array([ # add 0 column for intercept generated from RR, ignored bc coefficients for contrasts = 0
            [0, 1, -1, 0, 0],  # M vs P
            [0, -1, 0, 1, 0],  # C vs M
            [0, 0, -1, 1, 0],  # C vs P
            [0, -1, 0, 0, 1],  # L vs M
            [0, 0, -1, 0, 1],  # L vs P
            [0, 0, 0, -1, 1]   # L vs C
        ])

    elif paper == 'molina':
        contrast_matrix = np.array([[ 0, 1, -1	,1,-1],   # (M,C) vs (B,CC)
                                    [ 0, 1, 0,	-1,	0],   # (M vs C)
                                    [ 0, 0, 1	,0,-1]])  # (B vs CC)
  
    contrast_m = pd.DataFrame(contrast_matrix, columns=['intercept', 'M', 'P', 'C', 'L'])
    if new_order is not None:
        contrast_m = contrast_m[new_order]
    return contrast_m
        

In [52]:
def find_first_index(df, column, value, condition = None):
    if condition is not None:
        condition_column, condition_value = condition.split(' == ')
        return df[(df[column] == value) & (df[condition_column] == condition_value)].index[0]
    else:
        return (df[column] == value).idxmax()

In [53]:
qst = 'ssrs'
outcome = outcomes_dict[qst][0]
predictor = 'trtname'
data = data_dict[qst]['14']
rater = 'Teacher'

In [54]:
n_obs = data.shape[0]
alpha = 0.05
bonferroni = False
type_contrast = 'original'
formula =' ~ '.join((outcome, predictor))
#data = data[data['version_form'] == 'Teacher']

In [55]:
def perform_contrasts(data, outcome, predictor, type_contrast, alpha, bonferroni):
    
    if bonferroni : 
        alpha = alpha / n_obs 

    formula =' ~ '.join((outcome, predictor))
    # change when get access to CC data, careful with interpretations !
    contrasts_matrix = get_contrast_matrix(type_contrast)

    model = smf.mixedlm(formula, data = data, groups='src_subject_id')
    result = model.fit()

    new_order = [item.split('[')[0].lower() if 'Intercept' in item else item.split('[')[1][2] for item in result.model.exog_names]
    contrasts_matrix = contrasts_matrix[new_order]


    contrasts = result.t_test(contrasts_matrix)

    coef = contrasts.effect
    stderr = contrasts.sd
    z_scores = contrasts.tvalue
    p_values = contrasts.pvalue.round(4)
    conf_int = contrasts.conf_int()

    significant =  p_values < alpha

    comp_results_str = []

    for i, val in enumerate(significant): 

        if val and coef[i] > 0 :
            results_sign = ' < ' # this would be inverted if we were comparing the means, but we want to know which treatment DECREASE the outcome variable
        elif val and coef[i] < 0 :
            results_sign = ' > '
        else: 
            results_sign = ' ~ '
        comp = np.array(contrasts_matrix.columns[contrasts_matrix.iloc[i] != 0])
        comp_results_str.append(results_sign.join(comp))
    
    return {str(outcome): comp_results_str, 'p_val' + str(outcome): p_values}


In [56]:
data  = data_dict['ssrs']['14']

In [57]:
all_outcomes = np.concatenate(list(outcomes_dict.values()))
all_outcomes

array(['snainatx', 'snahypax', 'snaoddx', 'sspintx', 'ssptossx',
       'pcrcpax', 'pcrcprx'], dtype='<U8')

In [58]:
data_dict

{'snap': {'b':       src_subject_id interview_date  interview_age sex  snainatx  snahypax  \
  0              P1001     1997-06-28            156   M      0.11      0.00   
  1              P1001     1997-06-28            156   M      0.33      0.00   
  7              P1002     1997-06-03            120   M      1.00      0.50   
  8              P1002     1997-06-03            120   M      1.11      1.00   
  17             P1003     1995-06-27            108   M      2.44      2.60   
  ...              ...            ...            ...  ..       ...       ...   
  14509          P1864     1998-05-12             96   M      0.11      0.50   
  14513          P1865     1998-03-18             96   M      0.56      0.17   
  14518          P1866     1998-07-17             96   F      0.11      0.00   
  14524          P1867     1998-01-28             96   M      0.78      0.17   
  14532          P1868     1998-01-30             96   F      1.00      0.50   
  
         snaoddx  days_b

In [59]:
data

,src_subject_id,interview_date,interview_age,sex,days_baseline,site,version_form,ssptossx,sspintx,trtname
0,P1001,1997-06-27,156,M,0,1,Teacher,1.37,0.50,L
1,P1001,1998-10-08,171,M,468,1,Teacher,1.53,0.83,L
2,P1001,1998-10-08,171,M,468,1,Teacher,1.20,0.33,L
3,P1001,1998-10-15,172,M,475,1,Teacher,1.53,0.67,L
29,P1012,1997-07-02,144,M,0,3,Teacher,1.70,0.50,L
...,...,...,...,...,...,...,...,...,...,...
11012,P1867,1998-01-28,96,M,0,4,Teacher,1.33,0.67,L
11013,P1867,1999-03-05,109,M,401,4,Teacher,1.40,0.33,L
11016,P1868,1998-01-30,96,F,0,1,Teacher,1.67,0.83,L
11017,P1868,1998-11-23,106,F,297,1,Teacher,1.03,1.00,L


In [60]:
data[data['version_form'] == 'Teacher']

,src_subject_id,interview_date,interview_age,sex,days_baseline,site,version_form,ssptossx,sspintx,trtname
0,P1001,1997-06-27,156,M,0,1,Teacher,1.37,0.50,L
1,P1001,1998-10-08,171,M,468,1,Teacher,1.53,0.83,L
2,P1001,1998-10-08,171,M,468,1,Teacher,1.20,0.33,L
3,P1001,1998-10-15,172,M,475,1,Teacher,1.53,0.67,L
29,P1012,1997-07-02,144,M,0,3,Teacher,1.70,0.50,L
...,...,...,...,...,...,...,...,...,...,...
11012,P1867,1998-01-28,96,M,0,4,Teacher,1.33,0.67,L
11013,P1867,1999-03-05,109,M,401,4,Teacher,1.40,0.33,L
11016,P1868,1998-01-30,96,F,0,1,Teacher,1.67,0.83,L
11017,P1868,1998-11-23,106,F,297,1,Teacher,1.03,1.00,L


In [61]:
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'pc': ['pcrcpax', 'pcrcprx']}

In [62]:
data_dict.keys()

dict_keys(['snap', 'ssrs', 'pc'])

In [106]:
contrast_dict , p_vals_dict ={}, {}
contrast_list = []


for qst in data_dict.keys():
    data = data_dict[qst]['14']

    outcomes = outcomes_dict[qst]
    print(outcomes)
    version_form = raters if qst == 'snap' or qst == 'ssrs'  else None
    contrasts =[]

    print(qst, data.shape, outcomes, predictor, version_form, alpha)
    if version_form is not None:
        contrast = [[perform_contrasts(data[data['version_form'] == version_form[i]], out, predictor, type_contrast='original',  alpha= alpha, bonferroni=False) for out in outcomes]for i in range(len(raters))]
    else :
        contrast = [[perform_contrasts(data, out, predictor, type_contrast='original',  alpha= alpha, bonferroni=False) for out in outcomes]]
    print(contrast)
            

    contrast_dict[qst] = contrast
    contrast_list.append(contrast)
       # print(version_form)
    

['snainatx', 'snahypax', 'snaoddx']
snap (7502, 11) ['snainatx', 'snahypax', 'snaoddx'] trtname ['Teacher', 'Parent'] 0.05
[[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])}, {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])}, {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}], [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])}, {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])}, {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'], 'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.  

In [103]:
for i in range(len(qsts)):
    print(contrast_list[i])

[[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])}, {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])}, {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}], [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])}, {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])}, {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'], 'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.    , 0.    , 0.    ])}]]
[[{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'L > M', 'L > P', 'C > L'], 'p_valsspintx': array([0.2472

In [64]:

# qst , #0,1 rater, # 0,1 qst_var , # var or pval_val

In [65]:
all_outcomes_list = [['snainatx', 'snahypax', 'snaoddx'],['sspintx', 'ssptossx'],['pcrcpax', 'pcrcprx']]#, ['w1readb', 'w2math', 'w3spell']]
all_outcomes_list

[['snainatx', 'snahypax', 'snaoddx'],
 ['sspintx', 'ssptossx'],
 ['pcrcpax', 'pcrcprx']]

In [66]:
for qst in qsts:
    print(outcomes_dict[qst])

['snainatx', 'snahypax', 'snaoddx']
['sspintx', 'ssptossx']
['pcrcpax', 'pcrcprx']


In [109]:
contrast_list[0]

[[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
   'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])},
  {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'],
   'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])},
  {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
   'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}],
 [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
   'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
  {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
   'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
  {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'],
   'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.    , 0.    , 0.    ])}]]

In [68]:
contrast_dict['snap'][0][1]['snahypax']

['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L']

In [69]:
qsts

['snap', 'ssrs', 'pc']

In [74]:
contrast_dict

{'snap': [[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])},
   {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])},
   {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}],
  [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
   {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
   {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'],
    'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.    , 0.    , 0.    ])}]],
 'ssrs': [[{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 

In [75]:
results_table

,Unnamed: 0.1,Unnamed: 0,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F_value,replicated P_value,variable
0,0,0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,16.06,0.00,snainatx
1,1,1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,.56,1.47,0.11,snainatx
2,2,2,ADHD symptoms,Inattention,Teacher,Site,2.70,.02,0.29,0.92,snainatx
3,3,3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,11.42,0.00,snainatx
4,4,4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,.88,0.78,0.70,snainatx
5,5,5,ADHD symptoms,Inattention,Parent,Site,3.00,.02,0.48,0.79,snainatx
6,6,6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,11.42,0.00,snahypax
7,7,7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,.49,0.78,0.70,snahypax
8,8,8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,.02,0.48,0.79,snahypax
9,9,9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,21.57,0.00,snahypax


{'snap': [[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])},
   {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])},
   {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}],
  [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
   {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
   {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'],
    'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.    , 0.    , 0.    ])}]],
 'ssrs': [[{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 

In [110]:
for qst in qsts:
    for i in range(len(raters)):
        for j, var in enumerate(outcomes_dict[qst]):
            version_form = raters if qst == 'snap' or qst == 'ssrs' else None
            if version_form is not None:
                #print( var, contrast_dict[qst][i][j][var][0], contrast_dict[qst][i][j][var][1:3],  contrast_dict[qst][i][j][var][3:6],contrast_dict[qst][i][j]['p_val' +var])
         
                idx = find_first_index(results_table, 'variable', var, condition="Rater == {}".format(raters[i]))
                # results_table.loc[idx, 'M vs B'] =  contrast_dict[qst][i][j][var][0]
                # results_table.loc[idx: idx+1, 'C vs M and C vs B'] =  contrast_dict[qst][i][j][var][1:3]
                # results_table.loc[idx:idx +2, 'L vs (M, B, C)'] =  contrast_dict[qst][i][j][var][3:6]
            
            else:
                print(qst, var, contrast_dict[qst][0][j][var], contrast_dict[qst][0][j]['p_val' +var])
                idx = find_first_index(results_table,'variable', var)
                print(idx)
                print(results_table.loc[idx, 'Measure'])

pc pcrcpax ['M ~ P', 'C ~ M', 'C ~ P', 'L > M', 'L > P', 'C > L'] [0.4073 0.3736 0.0836 0.     0.     0.    ]
39
Power assertion
pc pcrcprx ['M ~ P', 'C ~ M', 'C ~ P', 'L ~ M', 'L ~ P', 'C ~ L'] [0.8651 0.5567 0.4461 0.723  0.8638 0.3181]
42
Personal closeness
pc pcrcpax ['M ~ P', 'C ~ M', 'C ~ P', 'L > M', 'L > P', 'C > L'] [0.4073 0.3736 0.0836 0.     0.     0.    ]
39
Power assertion
pc pcrcprx ['M ~ P', 'C ~ M', 'C ~ P', 'L ~ M', 'L ~ P', 'C ~ L'] [0.8651 0.5567 0.4461 0.723  0.8638 0.3181]
42
Personal closeness


In [137]:
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'pc': ['pcrcpax', 'pcrcprx']}

In [142]:
contrast_list

[[[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])},
   {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])},
   {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}],
  [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
   {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'],
    'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])},
   {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'],
    'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.    , 0.    , 0.    ])}]],
 [[{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'L > M', 'L > P'

In [144]:
snap_c = contrast_list[0]
print(snap_c)
snap_c_t = contrast_list[0][0]
snap_c_p = contrast_list[0][1]
print(snap_c_p, snap_c_t)
# for i, var in enumerate(snap_vars):
#     print(snap_c_t[i][var])
#     print(snap_c_t[i]['p_val' + var])

[[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([0.0003, 0.0539, 0.    , 0.    , 0.    , 0.    ])}, {'snahypax': ['M > P', 'C > M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnahypax': array([0.0103, 0.0033, 0.    , 0.    , 0.    , 0.    ])}, {'snaoddx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnaoddx': array([0.0172, 0.0748, 0.    , 0.    , 0.    , 0.    ])}], [{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([9.200e-03, 1.856e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])}, {'snahypax': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnahypax': array([7.000e-04, 5.591e-01, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00])}, {'snaoddx': ['M ~ P', 'C > M', 'C ~ P', 'L > M', 'L > P', 'C > L'], 'p_valsnaoddx': array([0.7046, 0.0198, 0.0506, 0.    , 0.    , 0.    ])}]]
[{'snainatx': ['M > P', 'C ~ M', 'C > P', 'L > M', 'L > P', 'C > L'], 'p_valsnainatx': array([9.200

In [153]:
all_outcomes_list

[['snainatx', 'snahypax', 'snaoddx'],
 ['sspintx', 'ssptossx'],
 ['pcrcpax', 'pcrcprx']]

In [179]:
results_table.loc[results_table['Measure'] == 'Power assertion', 'variable'] = pc_vars[0]
results_table.loc[results_table['Measure'] == 'Personal closeness'  ,  'variable'] = pc_vars[1]
results_table['M vs B'] = pd.NA
results_table['M vs B p_val'] = pd.NA
results_table['C vs (M,B)'] = pd.NA
results_table['C vs (M,B) p_val '] = pd.NA
results_table['L vs (M,B,C)'] = pd.NA
results_table['L vs (M,B,C) p_val'] = pd.NA
results_table


,Unnamed: 0.1,Unnamed: 0,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F_value,replicated P_value,variable,M vs B,M vs B p_val,"C vs (M,B)","C vs (M,B) p_val","L vs (M,B,C)","L vs (M,B,C) p_val"
0,0,0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,16.06,0.00,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,1,1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,.56,1.47,0.11,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2,2,ADHD symptoms,Inattention,Teacher,Site,2.70,.02,0.29,0.92,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,3,3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,11.42,0.00,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,4,4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,.88,0.78,0.70,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,5,5,ADHD symptoms,Inattention,Parent,Site,3.00,.02,0.48,0.79,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,6,6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,11.42,0.00,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,7,7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,.49,0.78,0.70,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,8,8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,.02,0.48,0.79,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,9,9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,21.57,0.00,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [187]:
j = 0 
for i,  qst in enumerate(['snap', 'ssrs']):
    qst_c = contrast_dict[qst]
    qst_c_t = qst_c[0]
    qst_c_p = qst_c[1]
    for i, var in enumerate(all_outcomes_list[i]):
        #teacher 
        
        cont = qst_c_t[i][var]
        p_val_c = qst_c_t[i]['p_val' +var]
        M_P = cont[0]
        M_P_p_val = p_val_c[0]
        
        C_MP = cont[1:3]
        C_MP_p_val = cont[1:3]
        
        L_CMP = cont[3:]
        L_CMP_p_val = cont[3:]
        
        #parent 
        cont_p = qst_c_p[i][var]
        p_val_c_p = qst_c_p[i]['p_val' +var]
        M_P_p = cont_p[0]
        M_P_p_val_p = p_val_c_p[0]
        
        C_MP_p = cont_p[1:3]
        C_MP_p_val_p = cont_p[1:3]
        
        L_CMP_p = cont_p[3:]
        L_CMP_p_val_p = cont_p[3:]
        print(p_val_c_p)
        # print(var, M_P, C_MP, L_CMP)
        # print(var, M_P_p_val, C_MP_p_val, L_CMP_p_val)
        first_index = results_table.index[results_table['variable'] == var].tolist()[0]
        print(first_index)
        print(results_table.loc[first_index, 'M vs B'])
        results_table.loc[first_index, 'M vs B'] = M_P
        results_table.loc[first_index:first_index+1, 'C vs (M,B)'] = C_MP
        results_table.loc[first_index:first_index+2, 'L vs (M,B,C)'] = L_CMP
        
        results_table.loc[first_index+3, 'M vs B'] = M_P_p
        results_table.loc[first_index+3:first_index+1+3, 'C vs (M,B)'] = C_MP_p
        results_table.loc[first_index+3:first_index+2+3, 'L vs (M,B,C)'] = L_CMP_p
        
        results_table.loc[first_index, 'M vs B p_val'] = M_P_p_val
        results_table.loc[first_index:first_index+1, 'C vs (M,B) p_val'] = C_MP_p_val
        results_table.loc[first_index:first_index+2, 'L vs (M,B,C) p_val'] = L_CMP_p_val
        
        results_table.loc[first_index+3, 'M vs B p_val'] = M_P_p_val_p
        results_table.loc[first_index+3:first_index+1+3, 'C vs (M,B) p_val'] = C_MP_p_val_p
        results_table.loc[first_index+3:first_index+2+3, 'L vs (M,B,C) p_val'] = L_CMP_p_val_p
        
    

[9.200e-03 1.856e-01 1.000e-04 0.000e+00 0.000e+00 0.000e+00]
0
M > P
[7.000e-04 5.591e-01 1.000e-04 0.000e+00 0.000e+00 0.000e+00]
6
M > P
[0.7046 0.0198 0.0506 0.     0.     0.    ]
15
M > P
[0.6233 0.7346 0.4037 0.     0.     0.    ]
24
M ~ P
[0.654  0.0197 0.0583 0.     0.     0.    ]
33
M < P


In [188]:
results_table

,Unnamed: 0.1,Unnamed: 0,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F_value,replicated P_value,variable,M vs B,M vs B p_val,"C vs (M,B)","C vs (M,B) p_val","L vs (M,B,C)","L vs (M,B,C) p_val","C vs (M,B) p_val"
0,0,0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,16.06,0.00,snainatx,M > P,0.0003,C ~ M,<NA>,L > M,L > M,C ~ M
1,1,1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,.56,1.47,0.11,snainatx,<NA>,<NA>,C > P,<NA>,L > P,L > P,C > P
2,2,2,ADHD symptoms,Inattention,Teacher,Site,2.70,.02,0.29,0.92,snainatx,<NA>,<NA>,<NA>,<NA>,C > L,C > L,nan
3,3,3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,11.42,0.00,snainatx,M > P,0.0092,C ~ M,<NA>,L > M,L > M,C ~ M
4,4,4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,.88,0.78,0.70,snainatx,<NA>,<NA>,C > P,<NA>,L > P,L > P,C > P
5,5,5,ADHD symptoms,Inattention,Parent,Site,3.00,.02,0.48,0.79,snainatx,<NA>,<NA>,<NA>,<NA>,C > L,C > L,nan
6,6,6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,11.42,0.00,snahypax,M > P,0.0103,C > M,<NA>,L > M,L > M,C > M
7,7,7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,.49,0.78,0.70,snahypax,<NA>,<NA>,C > P,<NA>,L > P,L > P,C > P
8,8,8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,.02,0.48,0.79,snahypax,<NA>,<NA>,<NA>,<NA>,C > L,C > L,nan
9,9,9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,21.57,0.00,snahypax,M > P,0.0007,C ~ M,<NA>,L > M,L > M,C ~ M


In [189]:
#results_table.to_csv(Path(derived_data, 'original_and_new_rr_14_contrasts.csv'))